In [2]:

import random
import numpy as np
from tqdm import tqdm
import json

random.seed(0)
# 直接从equal_split_word数据生成,考虑了coarse的情况
attr_dict_file = '../../data/new_data/equal_processed_data/dict/attr_relation_dict.json'
with open(attr_dict_file, 'r') as f:
    relation_dict = json.load(f)

# def get_negative_dict(attr_dict):
#         negative_dict = {}
#         for query, attr_list in attr_dict.items():
#             negative_dict[query] = {}
#             for attr in attr_list:
#                 l = attr_list.copy()
#                 l.remove(attr)
#                 negative_dict[query][attr] = l
#         return negative_dict
# negative_dict = get_negative_dict(attr_dict)

def generate_negative_data(input_data):
    rets = []
    for split_data in input_data:
        # print(len(split_data))
        split_rets = []
        for data in split_data:
            title = data['title']
            key_attr = data['key_attr']
            if not key_attr:
                continue
            match = data['match']
            vocab_split = data['vocab_split']
            del data['title_split']
            match['图文'] = 0
            # 先随机选取一个属性为负
            query = random.sample(list(key_attr.keys()), 1)[0]
            attr = key_attr[query]
            attr_list = random.sample(relation_dict[attr]['similar_attr'], 1)[0]
            if len(attr_list) == 1:
                new_attr = attr_list[0]
            else:
                new_attr = random.sample(attr_list, 1)[0]
            key_attr[query] = new_attr
            title = title.replace(attr, new_attr)
            idx = vocab_split.index(attr)
            vocab_split[idx] = new_attr
            match[query] = 0
            # 其余属性以一定概率变负
            for query, attr in key_attr.items():
                if match[query] != 0:
                    if random.random() < 0.4375: # 0.434
                        attr_list = random.sample(relation_dict[attr]['similar_attr'], 1)[0]
                        if len(attr_list) == 1:
                            new_attr = attr_list[0]
                        else:
                            new_attr = random.sample(attr_list, 1)[0]
                        key_attr[query] = new_attr
                        title = title.replace(attr, new_attr)
                        idx = vocab_split.index(attr)
                        vocab_split[idx] = new_attr
                        match[query] = 0
            # 隐藏属性
            if len(key_attr) >= 2: # 属性不少于两个
                flag_0 = 0
                flag_1 = 0
                for query, value in match.items(): # 有正有负
                    if query != '图文':
                        if value == 1:
                            flag_1 = 1
                        else:
                            flag_0 = 1
                if flag_0 and flag_1:
                    if random.random() < 0.3: # 一定概率进行隐藏
                        match_copy = match.copy()
                        for query, value in match_copy.items():
                            if query != '图文':
                                if value == 0:
                                    del match[query]
                                    del key_attr[query]
            data['title'] = title 
            data['key_attr'] = key_attr 
            data['match'] = match 
            data['vocab_split'] = vocab_split
            split_rets.append(data)
        rets.append(split_rets)
    return rets
                    

In [3]:
def generate_pos_data(input_data):
    rets = []
    for data in input_data:
        title = data['title']
        key_attr = data['key_attr']
        if not key_attr:
            continue
        match = data['match']
        vocab_split = data['vocab_split']
        del data['title_split']
        
        for query, attr in key_attr.items():
            if relation_dict[attr]['equal_attr']:
                if random.random() < 0.25: # 正例增强
                    new_attr = random.sample(relation_dict[attr]['equal_attr'], 1)[0]
                    key_attr[query] = new_attr
                    title = title.replace(attr, new_attr)
                    match[query] = 0.9
                    idx = vocab_split.index(attr)
                    vocab_split[idx] = new_attr
                    
        data['title'] = title 
        data['key_attr'] = key_attr 
        data['match'] = match 
        data['vocab_split'] = vocab_split
        rets.append(data)
    return rets

In [4]:
import json
import os
import copy

all_index = [0,1,2,3]

for pos_index in all_index:
    
    data_file = "../../data/new_data/divided/title/shuffle/fine5000.txt"
    origin_data = []
    with open(data_file, 'r') as f:
        for line in tqdm(f):
            item = json.loads(line)
            origin_data.append(item)
            
            
    split_data  = []
    item_len = len(origin_data)//4
    for i in range(0, len(origin_data), item_len):
        split_data.append(origin_data[i:i+item_len])
        
    split_data = np.array(split_data)
    reslut = np.array([None]*4)
    neg_index = all_index.copy()
    neg_index.remove(pos_index)
    print(f"pos_index:{pos_index} neg_index:{neg_index}")
    
    pos_data = split_data[pos_index]
    neg_data = split_data[neg_index]
    
    print(f"pos_len:{len(pos_data)} neg_len:{len(neg_data[0])}")
    
    
    pos_data = generate_pos_data(pos_data)
    neg_data = generate_negative_data(neg_data)
    
    reslut[pos_index] = pos_data
    reslut[neg_index] = neg_data
    
    save_result = []
    for split_data in reslut:
        for data in split_data:
            save_result.append(json.dumps(data, ensure_ascii=False)+'\n')
    
    save_path = os.path.join('../../output/fusion', f"pos_{pos_index}.txt")
    with open(save_path, 'w') as f:
        f.writelines(save_result)    

5000it [00:01, 2951.68it/s]


pos_index:0 neg_index:[1, 2, 3]
pos_len:1250 neg_len:1250


5000it [00:01, 2924.25it/s]


pos_index:1 neg_index:[0, 2, 3]
pos_len:1250 neg_len:1250


5000it [00:01, 2555.47it/s]


pos_index:2 neg_index:[0, 1, 3]
pos_len:1250 neg_len:1250


5000it [00:01, 3261.28it/s]


pos_index:3 neg_index:[0, 1, 2]
pos_len:1250 neg_len:1250


In [5]:
# no feat
for pos_index in range(4):
    save_path = os.path.join('../../output/fusion', f"pos_{pos_index}.txt")
    
    all_data = []
    with open(save_path, 'r') as f:
        for line in tqdm(f):
            data = json.loads(line)
            del data["feature"]
            all_data.append(json.dumps(data, ensure_ascii=False)+'\n')

    save_path = os.path.join('../../output/fusion/no_feat', f"pos_{pos_index}.txt")
    with open(save_path, 'w') as f:
        f.writelines(all_data)

5000it [00:01, 3482.14it/s]
5000it [00:01, 3462.10it/s]
5000it [00:01, 3499.42it/s]
5000it [00:01, 3465.56it/s]
